<h3>The purpose of this code is to normalise the linear theory Power Spectrum(PS) found from the primordial power spectrum evolved using a Transfer Function(TF). </h3>
<ul>
<li>The TF is sourced from Niladri's dataset [source file address: ./../Data/Transfer_camb_su_1024.txt]<br>
<li>Growth factor assumed is G(a) = a = 1/(1+z) | z=0 <br>
<li>The Normalization is done by comparing the sigma_8 value to the current (z=0). <br>
Sigma_8 is the variance in density perturbations smoothed over a scale of R=8 h^-1 MPc. <br>
(Detailed reference to using Top-Hat/Gaussian filter can be seen from Cooray & Sheth page 10) <br>
<li> Integration to be done using logarithmic spacing
<li>Cosmology in consideration is defined in the 2nd cell
<li> The Transfer Function is interpolated to more points to make th 2PC calculations later finer. for 50000 points, analytical 2PC works fine for R&lt;30MPc
</ul>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

R=8  # MPc h^-1

#File name that contains the transfer function
fname="./../Data/cambTf_Om0.276_Ol0.724_Ob0.045_h0.700_ns0.961.txt"
#fname="./../Data/Transfer_tinker-2005.txt"


In [2]:
# Define the Cosmology here

'''
# WMAP1 Cosmology
Om     0.270
Ob     0.046
OL     0.730
sigma_8     0.90
h           0.72
ns          0.99
'''

#'''
# Aseem's Cosmology
sigma_8=0.811
ns=0.961
h=0.7
Ob=0.045
Om=0.276
#'''

'''
# New Cosmology
sigma_8=
ns=
h=
'''

'\n# New Cosmology\nsigma_8=\nns=\nh=\n'

In [3]:
# Defines the smoothening window functions
def TopHat(k,R):
    return (3/np.power(k*R,3))*(np.sin(k*R)-(k*R)*np.cos(k*R))

def Gaussian(k,R):
    return np.exp(np.power((k*R),2)/-2.0)

In [4]:
# Loads transfer function data
data=np.loadtxt(fname)
k=np.array(data[:,0])
fb=Ob/Om
fcdm=1-fb
T_k=np.array(fcdm*data[:,1]+fb*data[:,2])

finesse=500000   # Finesse of 5e+5 works well for 2PC calculations for R<=50MPc  ==> Aseem
#k_finer=np.linspace(np.min(k),np.max(k),finesse)
k_finer=np.min(k)*np.exp(np.linspace(0.0,np.log(np.max(k)/np.min(k)),finesse))
T_k_finer=np.interp(k_finer,k,T_k)

In [5]:
# Computation of the variance
top=TopHat(k,R)
gaus=Gaussian(k,R)

top_f=TopHat(k_finer,R)
gaus_f=Gaussian(k_finer,R)

res1=np.power(T_k,2)*(k**ns)   # Tf^2 x k^n
res1=(k**3)*res1/(2*(np.pi**2))  # P(k)*k^3/2Pi^2 integrating with d(lnK)
#res1=(k**2)*res1/(2*np.pi**2)  # integrating with dK
res2=res1*(top**2)  # f1(k)=P(k)*K^3/2Pi^2 x W(kR)^2 (topHat)
res3=res1*(gaus**2)  # f2(k)=" " " " " " W(kR)^2 (Gaussian)

res1_f=np.power(T_k_finer,2)*(k_finer**ns)   # Tf^2 x k^n
res1_f=(k_finer**3)*res1_f/(2*(np.pi**2))  # P(k)*k^3/2Pi^2 integrating with d(lnK)
#res1_f=(k_finer**2)*res1_f/(2*np.pi**2)  # integrating with dK
res2_f=res1_f*(top_f**2)  # f1(k)=P(k)*K^3/2Pi^2 x W(kR)^2 (topHat)
res3_f=res1_f*(gaus_f**2)  # f2(k)=" " " " " " W(kR)^2 (Gaussian)

ansTop=np.trapz(res2,np.log(k))
#ansTop=np.trapz(res2,k)
ansGaus=np.trapz(res3,np.log(k))
#ansGaus=np.trapz(res3,k)

ansTop_f=np.trapz(res2_f,np.log(k_finer))
#ansTop_f=np.trapz(res2_f,k_finer)
ansGaus_f=np.trapz(res3_f,np.log(k_finer))
#ansGaus_f=np.trapz(res3_f,k_finer)

AT=(sigma_8**2)/ansTop
AG=(sigma_8**2)/ansGaus

AT_f=(sigma_8**2)/ansTop_f
AG_f=(sigma_8**2)/ansGaus_f

print(AT_f/AT)
print(AG_f/AG)

0.9953726071619561
0.9963533876282248


In [6]:
# Computing dimensionless powerspectrum
Dk_T=AT*res1
Dk_T_f=AT_f*res1_f
#print(np.interp(1.0,k,Dk_T)/np.interp(1.0,k_finer,Dk_T_f))

Dk_G=AG*res1
Dk_T_f=AT_f*res1_f

In [9]:
#scale=k_finer
#value=Dk_T_f
scale=k
value=Dk_T
#with open('NormalizedDimensionlessPS_finessed.txt','w') as f:
with open('NormalizedDimensionlessPS_atTFk.txt','w') as f:
#    f.write('# Aseem\'s Cosmology normalised linear theory Power Spectrum (Dimensionless) finessed over 50,000 k value interpolations over original Transfer Function input.\n')
    f.write('# Aseem\'s Cosmology normalised linear theory Power Spectrum (Dimensionless) at the same k values as input Transfer Function.\n')
    f.write('# k   Dk\n')
    for i in range(0,len(scale)):
        f.write(str(scale[i])+"\t\t"+str(value[i])+"\n")

In [7]:
R=pickle.load(open("./radii.p",'rb'))
xi_lin=[]
for r in R:
    xi_lin.append(np.trapz((Dk_T*TopHat(k,r)),np.log(k)))
pickle.dump(np.array(xi_lin),open("./linearCorr","wb"))